In [ ]:
# default_exp tef_core
from nbdev import *

# TEF core framework

> contains TEF framework

In [ ]:
#export
import math
import numpy as np
import xarray as xr
import logging
from tqdm import tqdm

In [ ]:
#export
logging.basicConfig(level=logging.INFO,
                    filename="logger.TEF.GETM",
                    filemode="w",
                    format="%(asctime)-15s %(levelname)-8s %(message)s")
logger = logging.getLogger(__name__)

In [ ]:
#export
class TEF_object():
    """Total Exchange Flow (TEF) Object used for calculating TEF properties"""
    def __init__(self, filename = None, ds = None, **kwargs):

        # What happens if no filename is provided
        if not filename:
            if ds is None:
                self.ds = None
            else:
                self.ds = ds
            return

        # continue with filename
        try:
            self.ds = None
            self.read(filename, **kwargs)
        except (OSError, IOError, RuntimeError):
            try:
                self.read(filename, **kwargs)
            except Exception:
                raise IOError("Unkown file format. Known formats are netcdf.")

        self.transport = None

    def __repr__(self):
        try:
            string = "xarray dataset with {} time steps \n" \
                     "Available fields: {}".format(
                self.timesteps, ", ".join(self.variables)
            )
        except AttributeError:
            string = "Empty TEF object \n" \
                     "Hint: Have you used read() to load data?"
        return string

    def __str__(self):
        return 'Class {}: \n'.format(self.__class__.__name__, self.ds)

    def __len__(self):
        return len(self.ds)

    def __getattr__(self, attr):
        if attr in self.__dict__:
            return getattr(self, attr)
        return getattr(self.ds, attr)

    def __getitem__(self, key):
        return self.ds[key]

    @property
    def timesteps(self):
        if len(self.ds.dims) != 3:
            logger.warning(
                "\nDimensions should be equal to 3, but they are not.\n"
                "You want ... "
            )
            return self.ds.dims[self._get_name_time()]
        return self.ds.dims[self.get_name_time()]

    @property
    def variables(self):
        return list(self.ds.data_vars)

    @property
    def dimensions(self):
        return list(self.ds.dims)

    def read(self, filename,  **kwargs):
        """Reads xarray dataset"""
        if self.ds is None:
            self.ds = xr.open_dataset(filename, **kwargs)
            logger.debug("read: {}".format(self.__str__))
        else:
            raise ValueError("TEF object is already set!")

    def set_up(self,
           time_name=None,
           longitude_name=None,
           latitude_name=None,
           depth_name=None):
        """Setup xarray.dataset and transposes dimensions into needed format"""
        # set dimensions
        if time_name is None:
            self._time_name = self._get_name_time()
        else:
            self._time_name = time_name
        if longitude_name is None:
            self._longitude_name = self._get_name_longitude()
            if self._longitude_name is None:
                self.ds = self.ds.expand_dims("lon")
                self._longitude_name = "lon"
                print("Created dummy dimension for longitude")
        else:
            self._longitude_name = longitude_name

        if latitude_name is None:
            self._latitude_name = self._get_name_latitude()
            if self._latitude_name is None:
                self.ds = self.ds.expand_dims("lat")
                self._latitude_name = "lat"
                print("Created dummy dimension for latitude")
        else:
            self._latitude_name = latitude_name

        if depth_name is None:
            self._depth_name = self._get_name_depth()
            if self._depth_name is None:
                self.ds = self.ds.expand_dims("depth")
                self._depth_name = "depth"
                print("Created dummy dimension for depth")
        else:
            self._depth_name = depth_name
        if time_name is None:
            self._time_name = self._get_name_time()
            if self._time_name is None:
                self.ds = self.ds.expand_dims("time")
                self._time_name = "time"
                print("Created dummy dimension for time")
        else:
            self._time_name = time_name
        
        # Transpose data
        self.ds = self.ds.transpose(self._time_name,
                                    self._depth_name,
                                    self._latitude_name,
                                    self._longitude_name)

    def _get_name_time(self, x = None):
        """
        check for 'time' dimension and return name
        """
        # check unit
        if x is not None:
            if isinstance(x, np.ndarray):
                logger.info('numpy array -> creating artificial time axis')
                return np.arange(x.shape[0]) 
        else:
            for dim in self.ds.dims:
                if (('units' in self.ds[dim].attrs and
                    'since' in self.ds[dim].attrs['units']) or
                    ('units' in self.ds[dim].encoding and
                     'since' in self.ds[dim].encoding['units']) or
                    dim in ['time']):
                    return dim
            # check dtype
            for dim in self.ds.variables:
                try:
                    var = self.ds[dim].data[0]
                except IndexError:
                    var = self.ds[dim].data
                if isinstance(var, np.datetime64):
                    return dim
            # no 'time' dimension found
            logger.warning(
                "\n 'time' dimension (dtype='datetime64[ns]') not found."
            )
            return None

    def _get_name_longitude(self):
        """
        check for 'longitude' dimension and return name
        """
        for dim in self.ds.dims:
            if (('units' in self.ds[dim].attrs and
               self.ds[dim].attrs['units'] in ['degree_east', 'degrees_east']) or
               dim in ['lon', 'longitude', 'x']):
                   return dim
        # no 'longitude' dimension found
        logger.warning(
            "\n 'longitude' dimension (unit='degrees_east') not found."
        )
        return None


    def _get_name_latitude(self):
        """
        check for 'latitude' dimension and return name
        """
        for dim in self.ds.dims:
            if (('units' in self.ds[dim].attrs  and
                self.ds[dim].attrs['units'] in ['degree_north', 'degrees_north']) or
                dim in ['lat', 'latitude', 'y']):
                return dim
        # no 'latitude' dimension found
        logger.warning(
            "\n 'latitude' dimension (unit='degrees_north') not found."
        )
        return None

    def _get_name_depth(self):
        """
        check for 'depth' dimension and return name
        """
        for dim in self.ds.dims:
            if (('units' in self.ds[dim].attrs  and
                self.ds[dim].attrs['units'] in ['vertical', 'level']) or
                dim in ['level', 'depth']):
                return dim
        # no 'latitude' dimension found
        logger.warning(
            "\n 'depth not found"
        )
        return None

    from pyTEF.calc import convert_q_to_Q
    from pyTEF.calc import sort_1dim
    from pyTEF.calc import sort_2dim
    from pyTEF.calc import calc_bulk_values

In [ ]:
show_doc(TEF_object)

<h2 id="TEF_object" class="doc_header"><code>class</code> <code>TEF_object</code><a href="" class="source_link" style="float:right">[source]</a></h2>

> <code>TEF_object</code>(**`filename`**=*`None`*, **`ds`**=*`None`*, **\*\*`kwargs`**)

Total Exchange Flow (TEF) Object used for calculating TEF properties

The user can either provide the path to a netCDF-dataset using `filename`, which will then be loaded as a xarray dataset. If a pre-loaded xarray dataset is already present, the user can simply assign this to the `TEF_object` by using `ds`.

In [ ]:
show_doc(TEF_object.timesteps)

<h4 id="TEF_object.timesteps" class="doc_header"><code>TEF_object.timesteps</code><a href="" class="source_link" style="float:right">[source]</a></h4>



Returns the number of timesteps of the given.

In [ ]:
show_doc(TEF_object.read)

<h4 id="TEF_object.read" class="doc_header"><code>TEF_object.read</code><a href="__main__.py#L69" class="source_link" style="float:right">[source]</a></h4>

> <code>TEF_object.read</code>(**`filename`**, **\*\*`kwargs`**)

Reads xarray dataset

Is called during the initialization of the the `TEF_object` when a `filename` is provided.

In [ ]:
show_doc(TEF_object.set_up)

<h4 id="TEF_object.set_up" class="doc_header"><code>TEF_object.set_up</code><a href="__main__.py#L77" class="source_link" style="float:right">[source]</a></h4>

> <code>TEF_object.set_up</code>(**`time_name`**=*`None`*, **`longitude_name`**=*`None`*, **`latitude_name`**=*`None`*, **`depth_name`**=*`None`*)

Setup xarray.dataset and transposes dimensions into needed format

`TEF_object.set_up` tries to automatically find the corresponding dimensions. If no 4D array is provided, dummy dimensions will be created. However, sometimes the naming of the dimensions deviates from common dimension descriptions. If so, the explicit dimension names can be provided as well. The function automatically reorders the dimensions into `time, z, y, x`.

In [ ]:
# Try using uncommon dimensions
import xarray as xr
import numpy as np 

lat = np.arange(-180, 180)
lon = 37
depth = np.arange(0, 200)
time = 2
dummy_var = np.zeros((lat.size, depth.size))

ds = xr.Dataset({'dummy': (["lAtiTude", "dePht"], dummy_var)},
                coords = {'lAtiTude': (["lAtiTude"], lat),
                          'dePht': (["dePht"], depth)})

tef = TEF_object(ds=ds)
tef.set_up(latitude_name="lAtiTude", depth_name="dePht")
tef.ds

Created dummy dimension for longitude
Created dummy dimension for time


<xarray.Dataset>
Dimensions:   (dePht: 200, lAtiTude: 360, lon: 1, time: 1)
Coordinates:
  * lAtiTude  (lAtiTude) int64 -180 -179 -178 -177 -176 ... 175 176 177 178 179
  * dePht     (dePht) int64 0 1 2 3 4 5 6 7 ... 192 193 194 195 196 197 198 199
Dimensions without coordinates: lon, time
Data variables:
    dummy     (time, dePht, lAtiTude, lon) float64 0.0 0.0 0.0 ... 0.0 0.0 0.0

In [ ]:
show_doc(TEF_object._get_name_time)

<h4 id="TEF_object._get_name_time" class="doc_header"><code>TEF_object._get_name_time</code><a href="__main__.py#L129" class="source_link" style="float:right">[source]</a></h4>

> <code>TEF_object._get_name_time</code>()

check for 'time' dimension and return name

In [ ]:
show_doc(TEF_object._get_name_depth)

<h4 id="TEF_object._get_name_depth" class="doc_header"><code>TEF_object._get_name_depth</code><a href="__main__.py#L186" class="source_link" style="float:right">[source]</a></h4>

> <code>TEF_object._get_name_depth</code>()

check for 'depth' dimension and return name

In [ ]:
show_doc(TEF_object._get_name_longitude)

<h4 id="TEF_object._get_name_longitude" class="doc_header"><code>TEF_object._get_name_longitude</code><a href="__main__.py#L155" class="source_link" style="float:right">[source]</a></h4>

> <code>TEF_object._get_name_longitude</code>()

check for 'longitude' dimension and return name

In [ ]:
show_doc(TEF_object._get_name_latitude)

<h4 id="TEF_object._get_name_latitude" class="doc_header"><code>TEF_object._get_name_latitude</code><a href="__main__.py#L171" class="source_link" style="float:right">[source]</a></h4>

> <code>TEF_object._get_name_latitude</code>()

check for 'latitude' dimension and return name